As a data analyst, you have been given a task to monitor and report US domestic airline flights performance. Goal is to analyze the performance of the reporting airline to improve fight reliability thereby improving customer relaibility.

Below are the key report items,

*   Yearly airline performance report 
*   Yearly average flight delay statistics


## Loading the Dataset

In [15]:
import pandas as pd
airline_data =  pd.read_csv('airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})
# Displaying the first two rows
airline_data.iloc[:2,30:50]

,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode
0,1330,1330.0,0.0,0.0,0.0,0.0,1300-1359,8.0,1338.0,1415.0,5.0,1426,1420.0,-6.0,0.0,0.0,-1.0,1400-1459,0.0,NaN
1,1301,1255.0,-6.0,0.0,0.0,-1.0,1300-1359,9.0,1304.0,1358.0,13.0,1423,1411.0,-12.0,0.0,0.0,-1.0,1400-1459,0.0,NaN


In [31]:
airline_data.shape

(27000, 110)

In [32]:
airline_data.columns

Index(['Unnamed: 0', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline',
       'IATA_CODE_Reporting_Airline',
       ...
       'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID',
       'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime',
       'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum'],
      dtype='object', length=110)

## Importing the required libraries

In [ ]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
from flask import request

In [11]:
# Creating the dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True


# List of years 
year_list = [i for i in range(2005, 2021, 1)]

# creating a function to get each filtered data for creating yearly airline performance report 

def compute_data_choice_1(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    # Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data

# creating a function to get each filtered data for creating yearly airline delay report

def compute_data_choice_2(df):
    # Compute delay averages
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


# Application layout
app.layout = html.Div(children= [html.H1('US Domestic Airline Flights Performance',
                                         style = {'textAlign': 'center',
                                                 'color': '#503D36',
                                                 'font-size':24}),

                                # Dropdown creation
                                # Creating an outer division 
                                html.Div([
                                    # Adding a division
                                    html.Div([
                                        # Creating a division for adding dropdown helper text for report type
                                        html.Div(
                                            [
                                            html.H2('Report Type:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        #  Add a dropdown
                                         dcc.Dropdown(id='input-type',
                                             options= [
                                                 {'label': 'Yearly Airline Performance Report', 'value': 'OPT1'},
                                                 {'label': 'Yearly Airline Delay Report', 'value': 'OPT2'}
                                             ],
                                             placeholder='Select a report type',
                                             style= {'width':'80%','padding':'3px','font-size':'20px','textAlign':'center',}),
                                 # Placing them next to each other using the division style
                                    ], style={'display':'flex'}),
                                    
                                   # Adding a next division 
                                   html.Div([
                                       # Creating a division for adding dropdown helper text for choosing year
                                        html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-year', 
                                                     # Updating dropdown values using list comphrehension
                                                     options=[{'label': i, 'value': i} for i in year_list],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            # Placing them next to each other using the division style
                                            ], style={'display': 'flex'}),  
                                          ]),
                                
                                # Adding Computed graphs
                                html.Div([ ], id='plot1'),
    
                                html.Div([
                                        html.Div([ ], id='plot2'),
                                        html.Div([ ], id='plot3')
                                ], style={'display': 'flex'}),
                                
                                # Add a division with two empty divisions inside.
                                 html.Div([
                                     html.Div([ ], id = 'plot4'),
                                     html.Div([ ], id = 'plot5')],
                                     style = {'display': 'flex'})
                                ])

# Callback function definition

@app.callback( [Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
                Output(component_id='plot3', component_property='children'),
                Output(component_id='plot4', component_property='children'),
                Output(component_id='plot5', component_property='children')],
               [Input(component_id='input-type', component_property='value'),
                Input(component_id='input-year', component_property='value')],
               # Holding output state till user enters all the form information. In this case, it will be chart type and year
               [State("plot1", 'children'), State("plot2", "children"),
                State("plot3", "children"), State("plot4", "children"),
                State("plot5", "children")
               ])
# Creating a function to return the graphs 
def get_graph(chart, year, children1, children2, c3, c4, c5):
      
        # Select data
        df =  airline_data[airline_data['Year']==int(year)]
       
        if chart == 'OPT1':
            # Getting the required data for creating graph 
            bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df)
            
            #  A bar graph showing the number of flights under different cancellation categories
            bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
            
            # A line chart for Average flight time by reporting airline
            line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline', title='Average Monthly flight time (minutes) by airline')
            
            # A pie chart showing the percentage of diverted airport landings per reporting airline
            pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
            
            # A map showing the number of flights flying from each state using choropleth
            map_fig = px.choropleth(map_data,  # Input data
                    locations='OriginState', 
                    color='Flights',  
                    hover_data=['OriginState', 'Flights'], 
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale='GnBu',
                    range_color=[0, map_data['Flights'].max()]) 
            map_fig.update_layout(
                    title_text = 'Number of flights from origin state', 
                    geo_scope='usa')
            
            # A tree map of the number of flights flying to each state from each reporting airline
            tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'], 
                      values='Flights',
                      color='Flights',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state'
                )
           # Returned dcc.Graph component to the empty division
            return [dcc.Graph(figure=tree_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=map_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)
                   ]
        else:
            # This covers chart type 2 
            # Computing required information for creating graph from the data
            avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
            
            # Create graph
            carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
            weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
            nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
            sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
            late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
            return[dcc.Graph(figure=carrier_fig), 
                   dcc.Graph(figure=weather_fig), 
                   dcc.Graph(figure=nas_fig), 
                   dcc.Graph(figure=sec_fig), 
                   dcc.Graph(figure=late_fig)]
    
# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False, port = 2221)


Dash is running on http://localhost:2221/



 * Running on http://localhost:2221/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2023 14:13:30] "GET /_alive_70c58b49-1a22-4592-858c-100b7a373430 HTTP/1.1" 200 -


127.0.0.1 - - [26/Mar/2023 14:13:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:31] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'



127.0.0.1 - - [26/Mar/2023 14:13:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:50] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/Mar/2023 14:13:50] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'



127.0.0.1 - - [26/Mar/2023 14:13:52] "POST /_dash-update-component HTTP/1.1" 500 -


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'



127.0.0.1 - - [26/Mar/2023 14:13:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:58] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2023 14:13:59] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
